In [2]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)\s+batch_size\s+(?P<batch_size>[\d.]+)"


  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)
  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
      'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
      'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
      'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_walltime', "batch_size"
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'h100_batch'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
ncore = str(1)
!OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)
df

,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_walltime,batch_size
0,43,6,1,41,20,2,18.334693,0.00106,0.001046,0.002105,7.712960,1.993664,9.706624,461.350531,1
1,43,6,1,42,21,2,9.016269,0.00106,0.001046,0.002105,0.607904,0.316000,0.923904,3.208632,2
2,43,6,1,42,21,2,8.989168,0.00106,0.001046,0.002105,0.839328,0.305536,1.144864,3.313923,4
3,43,6,1,42,21,2,8.964976,0.00106,0.001046,0.002105,1.263776,0.304416,1.568192,3.764851,8
4,43,6,1,42,21,2,9.022610,0.00106,0.001046,0.002105,2.100224,0.304096,2.404320,4.601725,16
5,43,6,1,42,21,2,9.006730,0.00106,0.001046,0.002105,3.794176,0.305184,4.099360,6.251141,32
6,43,6,1,42,21,2,9.041136,0.00106,0.001046,0.002105,7.168224,0.304800,7.473024,9.628744,64
7,43,6,1,42,21,2,9.030965,0.00106,0.001046,0.002105,14.024032,0.306880,14.330912,16.519205,128
8,43,6,1,42,21,2,9.078050,0.00106,0.001046,0.002105,27.867136,0.312160,28.179296,30.302901,256
9,43,6,1,42,21,2,9.086624,0.00106,0.001046,0.002105,55.519810,0.318912,55.838722,58.027328,512


In [4]:
!nproc
!lscpu
!nvidia-smi

208


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   208
  On-line CPU(s) list:    0-207
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) Platinum 8470
    CPU family:           6
    Model:                143
    Thread(s) per core:   2
    Core(s) per socket:   52
    Socket(s):            2
    Stepping:             8
    Frequency boost:      enabled
    CPU max MHz:          2001.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4000.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc 